### Leitura de Bibliotecas

In [24]:
import requests
import pandas as pd

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


### Api com os dados dos deputados

In [ ]:
##Para verificar com mais detalhes a descrição dos campos acesse: https://dadosabertos.camara.leg.br/swagger/api.html

url = 'https://dadosabertos.camara.leg.br/api/v2/deputados'

response = requests.get(url)
if response.status_code == 200:
    #Tranformando o retorno da api para dataframe
    df_deputados = pd.DataFrame(response.json()['dados'])
    print(df_deputados.shape)
    
else:
    print(f'Erro ao obter lista de deputados: {response.status_code}')

In [33]:
#Exibimos os 5 primeiros registros
df_deputados.head()

,id,uri,nome,siglaPartido,uriPartido,siglaUf,idLegislatura,urlFoto,email
0,220593,https://dadosabertos.camara.leg.br/api/v2/depu...,Abilio Brunini,PL,https://dadosabertos.camara.leg.br/api/v2/part...,MT,57,https://www.camara.leg.br/internet/deputado/ba...,dep.abiliobrunini@camara.leg.br
1,204379,https://dadosabertos.camara.leg.br/api/v2/depu...,Acácio Favacho,MDB,https://dadosabertos.camara.leg.br/api/v2/part...,AP,57,https://www.camara.leg.br/internet/deputado/ba...,dep.acaciofavacho@camara.leg.br
2,220714,https://dadosabertos.camara.leg.br/api/v2/depu...,Adail Filho,REPUBLICANOS,https://dadosabertos.camara.leg.br/api/v2/part...,AM,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adailfilho@camara.leg.br
3,221328,https://dadosabertos.camara.leg.br/api/v2/depu...,Adilson Barroso,PL,https://dadosabertos.camara.leg.br/api/v2/part...,SP,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adilsonbarroso@camara.leg.br
4,204560,https://dadosabertos.camara.leg.br/api/v2/depu...,Adolfo Viana,PSDB,https://dadosabertos.camara.leg.br/api/v2/part...,BA,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adolfoviana@camara.leg.br


### Api de gastos dos deputados

In [34]:
gastos = []

for indice, deputado in df_deputados.iterrows():
    id = deputado['id']
    nome = deputado['nome']
    url_gastos = f'https://dadosabertos.camara.leg.br/api/v2/deputados/{id}/despesas?ano=2023&ordem=ASC'
    #print(url_gastos)
    response_gastos = requests.get(url_gastos)
    if response_gastos.status_code == 200:
        gasto = pd.DataFrame(response_gastos.json()['dados'])
        gasto['id'] = id
        gastos.append(gasto)

    else:
        print(f'Erro ao obter gastos do deputado {nome}')

# Concatena todos os DataFrames armazenados na lista 'gastos' em um único DataFrame
df_gastos = pd.concat(gastos)

In [35]:
df_gastos.shape

(7576, 18)

In [38]:
df_gastos.head()

,ano,mes,tipoDespesa,codDocumento,tipoDocumento,codTipoDocumento,dataDocumento,numDocumento,valorDocumento,urlDocumento,nomeFornecedor,cnpjCpfFornecedor,valorLiquido,valorGlosa,numRessarcimento,codLote,parcela,id
0,2023.0,9.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7610066.0,Nota Fiscal Eletrônica,4.0,2023-09-19,35537,52.98,http://www.camara.leg.br/cota-parlamentar/nota...,3XIS COMERCIO VAREJISTA E ATACADISTA DE PAPELA...,42999796000188,52.98,0.0,,1969221.0,0.0,220593
1,2023.0,11.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7657491.0,Nota Fiscal,0.0,2023-11-27,01,67.30,https://www.camara.leg.br/cota-parlamentar/doc...,AGUAS CUIABA S.A,14995581000153,67.30,0.0,,1996163.0,0.0,220593
2,2023.0,5.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7552621.0,Nota Fiscal,0.0,2023-05-05,11533052023001,43.20,https://www.camara.leg.br/cota-parlamentar/doc...,AGUAS CUIABA S.A,14995581000153,43.20,0.0,,1938600.0,0.0,220593
3,2023.0,6.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7587236.0,Recibos/Outros,1.0,2023-06-05,11533062023001,43.20,https://www.camara.leg.br/cota-parlamentar/doc...,AGUAS CUIABA S.A,14995581000153,43.20,0.0,,1957493.0,0.0,220593
4,2023.0,7.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7587239.0,Nota Fiscal,0.0,2023-07-05,11533072023001,43.20,https://www.camara.leg.br/cota-parlamentar/doc...,AGUAS CUIABA S.A,14995581000153,43.20,0.0,,1957494.0,0.0,220593


Fazendo o merge do dataframe de deputados com o dataframe de gastos

In [44]:
# Especifique as colunas que você deseja no resultado final
colunas_deputados = ['id', 'nome', 'siglaPartido', 'siglaUf']
colunas_gastos = ['id', 'ano', 'mes', 'tipoDespesa','nomeFornecedor', 'cnpjCpfFornecedor', 
                  'valorDocumento', 'valorGlosa', 'valorLiquido']

# Mesclando os DataFrames com base na coluna 'id'
df_gastos_x_deputado = pd.merge(df_deputados[colunas_deputados], df_gastos[colunas_gastos], on='id', how='inner')

In [45]:
df_gastos_x_deputado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7576 entries, 0 to 7575
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 7576 non-null   int64  
 1   nome               7576 non-null   object 
 2   siglaPartido       7576 non-null   object 
 3   siglaUf            7576 non-null   object 
 4   ano                7576 non-null   float64
 5   mes                7576 non-null   float64
 6   tipoDespesa        7576 non-null   object 
 7   nomeFornecedor     7576 non-null   object 
 8   cnpjCpfFornecedor  7576 non-null   object 
 9   valorDocumento     7576 non-null   float64
 10  valorGlosa         7576 non-null   float64
 11  valorLiquido       7576 non-null   float64
dtypes: float64(5), int64(1), object(6)
memory usage: 710.4+ KB


In [46]:
#Convertendo o mês e o ano e float para int
df_gastos_x_deputado = df_gastos_x_deputado.astype({'mes': int, 'ano': int})

In [47]:
df_gastos_x_deputado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7576 entries, 0 to 7575
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 7576 non-null   int64  
 1   nome               7576 non-null   object 
 2   siglaPartido       7576 non-null   object 
 3   siglaUf            7576 non-null   object 
 4   ano                7576 non-null   int32  
 5   mes                7576 non-null   int32  
 6   tipoDespesa        7576 non-null   object 
 7   nomeFornecedor     7576 non-null   object 
 8   cnpjCpfFornecedor  7576 non-null   object 
 9   valorDocumento     7576 non-null   float64
 10  valorGlosa         7576 non-null   float64
 11  valorLiquido       7576 non-null   float64
dtypes: float64(3), int32(2), int64(1), object(6)
memory usage: 651.2+ KB


In [48]:
df_gastos_x_deputado.head(5)

,id,nome,siglaPartido,siglaUf,ano,mes,tipoDespesa,nomeFornecedor,cnpjCpfFornecedor,valorDocumento,valorGlosa,valorLiquido
0,220593,Abilio Brunini,PL,MT,2023,9,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,3XIS COMERCIO VAREJISTA E ATACADISTA DE PAPELA...,42999796000188,52.98,0.0,52.98
1,220593,Abilio Brunini,PL,MT,2023,11,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,AGUAS CUIABA S.A,14995581000153,67.30,0.0,67.30
2,220593,Abilio Brunini,PL,MT,2023,5,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,AGUAS CUIABA S.A,14995581000153,43.20,0.0,43.20
3,220593,Abilio Brunini,PL,MT,2023,6,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,AGUAS CUIABA S.A,14995581000153,43.20,0.0,43.20
4,220593,Abilio Brunini,PL,MT,2023,7,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,AGUAS CUIABA S.A,14995581000153,43.20,0.0,43.20
